# Bag of Words
1. 통계와 머신러닝을 활용한 방법
2. 인공 신경망을 활용한 방법
- 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법
- BoW 생성 과정
 1. 우선, 각 단어에 고유한 정수 인덱스를 부여합니다.
 2. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듭니다.
- 참고 : https://wikidocs.net/22650

In [1]:
# 문장과 그 BoW 샘플 (확인용)

doc1 = 'John likes to watch movies. Mary likes movies too.'
BoW1 = {"John":1, "likes":2, "to":1, "watch":1, "movies":2, "Mary":1, "too":1}

doc2 = 'Mary also likes to watch football games.'
BoW2 = {"Mary":1, "also":1, "likes":1, "to":1, "watch":1, "football":1, "games":1}

doc3 = 'John likes to watch movies. Mary likes movies too. Mary also likes to watch football games.'
BoW3 = {"John":1, "likes":3, "to":2, "watch":2, "movies":2, "Mary":2, "too":1, "also":1, "football":1, "games":1}

## Keras Tokenizer를 활용한 BoW

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

In [3]:
def print_bow(sentence):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(sentence)  # 단어장 생성
  bow = dict(tokenizer.word_counts)  # 각 단어와 각 단어의 빈도를 bow에 저장
  print("Bag of Words :", bow)  # BoW 출력
  print('단어장(vocabulary)의 크기 :', len(tokenizer.word_counts))  # 중복을 제거한 단어들의 개수

print_bow(sentence)

Bag of Words : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(vocabulary)의 크기 : 10


## scikit-learn CountVectorizer를 활용한 BoW

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()
print('Bag of Words :', vector.fit_transform(sentence).toarray())  # 코퍼스로부터 각 단어의 빈도수 기록
print('각 단어의 인덱스 :', vector.vocabulary_)  # 각 단어의 인덱스가 어떻게 부여되는지 확인

Bag of Words : [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


In [6]:
print('단어장(vocabulary)의 크기 :', len(vector.vocabulary_))

단어장(vocabulary)의 크기 : 10


# 문서 단어 행렬(Document-Term Matrix, DTM)
- 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것

문서 1 : I like dog  
문서 2 : I like cat  
문서 3 : I like cat I like cat

|-|cat|dog|I|like|
|---|---|---|---|---|
|문서1|0|1|1|1|
|문서2|1|0|1|1|
|문서3|2|0|2|2|

In [19]:
import pandas as pd

content = [[0,1,1,1],[1,0,1,1],[2,0,2,2,]]
df = pd.DataFrame(content)
df.index = ['문서1', '문서2', '문서3']
df.columns = ['cat', 'dog', 'I', 'like']
df

,cat,dog,I,like
문서1,0,1,1,1
문서2,1,0,1,1
문서3,2,0,2,2


In [7]:
import numpy as np
from numpy import dot 
from numpy.linalg import norm

doc1 = np.array([0, 1, 1, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [8]:
print(cos_sim(doc1, doc2)) 
print(cos_sim(doc1, doc3))
print(cos_sim(doc2, doc3))

0.6666666666666667
0.6666666666666667
1.0000000000000002


- DTM에서는 코사인 유사도는 0이상 1이하의 값을 가지고, 값이 1에 가까울수록 유사도 높다 판단

## scikit-learn CountVectorizer를 활용한 DTM 구현

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
          'John likes to watch movies',
          'Mary likes movies too',
          'Mary also likes to watch football ga'
]

vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())  # 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.vocabulary_)  # 각 단어의 인덱스가 어떻게 부여되는지 확인

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'ga': 2}


# TF-IDF (Term Frequency-Inverse Document Frequency)
- 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법
- 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단
- 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단

In [13]:
from math import log
import pandas as pd

docs = [
        'John likes to watch movies and Mary likes movies too',
        'James likes to watch TV',
        'Mary also likes to watch football games',
]

In [14]:
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print('단어장의 크기 :', len(vocab))
print(vocab)

단어장의 크기 : 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [15]:
N = len(docs)
N

3

In [16]:
def tf(t,d):
  return d.count(t)

def idf(t):
  df = 0
  for doc in docs:
    df += t in doc
  return log(N/(df + 1)) + 1

def tfidf(t, d):
  return tf(t,d) * idf(t)

## TF 함수를 사용하여 DTM을 만들어 보자.

In [17]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]

    result[-1].append(tf(t,d))

tf_ = pd.DataFrame(result, columns=vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


In [18]:
result = []
for j in range(len(vocab)):
  t = vocab[j]
  result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


- watch, too, to는 중요도가 낮다.

### TF-IDF 행렬을 출력 DTM에 있는 각 단어의 TF에 각 단어의 IDF를 곱해 준 값

In [20]:
result = []
for i in range(N):
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]

    result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns= vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


'John likes to watch movies and Mary likes movies too',  
'James likes to watch TV',  
'Mary also likes to watch football games'

## scikit-learn TFidVectorizer 활용

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
          'John likes to watch movies and Mary likes movies too',
          'James likes to watch TV',
          'Mary also likes to watch football games',
]

tfidfv = TfidfVectorizer().fit(corpus)
vocab = list(set(tfidfv.vocabulary_.keys()))
vocab.sort()

tfidf_ = pd.DataFrame(tfidfv.transform(corpus).toarray(), columns=vocab)
tfidf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634
